# 微調 Open AI 模型

本筆記本根據 Open AI 官方 [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) 文件的最新指引編寫。

微調可以透過加入更多針對你應用場景的數據和背景，重新訓練基礎模型，從而提升模型在特定用途下的表現。要留意，像 _few shot learning_（少量示例學習）和 _retrieval augmented generation_（檢索增強生成）這類 prompt engineering 技巧，可以在預設提示中加入相關數據來提升質素。不過，這些方法都受限於目標基礎模型的最大 token 視窗大小。

透過微調，我們其實是用所需數據重新訓練模型本身（可以用遠多於 token 視窗能容納的例子），並部署一個 _自訂_ 版本的模型，推理時就不用再提供示例。這不但令我們設計 prompt 時更有彈性（token 視窗可以用來做其他事），還有機會降低成本（因為推理時要傳送給模型的 token 數量減少）。

微調分為四個步驟：
1. 準備訓練數據並上載。
1. 執行訓練作業，獲得微調後的模型。
1. 評估微調模型並反覆優化質素。
1. 當滿意後，部署微調模型作推理用途。

要注意，並非所有基礎模型都支援微調——[請參考 OpenAI 文件](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) 以獲取最新資訊。你亦可以對已經微調過的模型再進行微調。在本教學中，我們會以 `gpt-35-turbo` 作為目標基礎模型進行微調。

---


### 步驟 1.1：準備你的數據集

我哋一齊嚟建立一個 chatbot，幫你用打油詩嘅方式回答元素週期表相關嘅問題，等你可以更容易理解。喺呢個簡單教學入面，我哋只會建立一個數據集，用幾個示例回應去訓練模型，展示數據應該有嘅格式。現實應用上，你需要建立一個包含更多例子嘅數據集。如果有現成嘅開放數據集（適合你嘅應用範疇），你都可以用嚟重新格式化，方便微調使用。

因為我哋專注於 `gpt-35-turbo`，而且只需要單輪回應（chat completion），所以可以用[呢個建議格式](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst)去建立例子，符合 OpenAI chat completion 嘅要求。如果你預期會有多輪對話內容，就要用[多輪對話格式](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst)，當中會有個 `weight` 參數，標示邊啲訊息要用（或者唔用）嚟做微調。

我哋今次教學會用較簡單嘅單輪格式。數據會用 [jsonl 格式](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst)，每行一個紀錄，每個紀錄都係 JSON 格式嘅 object。下面嘅片段展示咗兩個紀錄作為示例——完整示例集（10 個例子）可以睇 [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl)，我哋會用嚟做微調教學。**注意：**每個紀錄 _必須_ 只用一行表示（唔可以分多行，好似一般 JSON 文件咁）。

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

現實應用上，你需要一個大得多嘅例子集先可以有好效果——你要喺回應質素同微調所需時間／成本之間作取捨。我哋用細數據集，係為咗可以快啲完成微調，方便示範流程。想睇更複雜嘅微調教學，可以參考[呢個 OpenAI Cookbook 範例](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst)。


### 步驟 1.2 上載你的數據集

用 Files API 上載數據 [詳情請參考這裡](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file)。請注意，要運行以下程式碼，你必須先完成以下步驟：
 - 已安裝 `openai` Python 套件（請確保使用版本 >=0.28.0，以獲得最新功能）
 - 已將 `OPENAI_API_KEY` 環境變數設置為你的 OpenAI API 金鑰
如需了解更多，請參閱本課程提供的[設定指南](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst)。

現在，運行以下程式碼，從你的本地 JSONL 檔案建立一個可供上載的檔案。


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### 步驟 2.1：用 SDK 建立微調任務


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### 步驟 2.2：檢查工作狀態

你可以用 `client.fine_tuning.jobs` API 做以下幾件事：
- `client.fine_tuning.jobs.list(limit=<n>)` - 列出最近 n 個微調工作
- `client.fine_tuning.jobs.retrieve(<job_id>)` - 查看某個微調工作的詳細資料
- `client.fine_tuning.jobs.cancel(<job_id>)` - 取消一個微調工作
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - 列出該工作最多 n 個事件
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

這個流程的第一步是 _驗證訓練檔案_，確保資料格式正確。


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### 步驟 2.3：追蹤事件以監察進度


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### 步驟 2.4：在 OpenAI 控制台查看狀態


你亦可以到 OpenAI 網站，進入平台的 _Fine-tuning_ 部分，查閱狀態。你可以睇到目前工作的狀態，亦可以追蹤過往執行紀錄。在呢張截圖入面，你會見到之前一次執行失敗，而第二次就成功咗。補充說明，第一次失敗係因為用咗格式錯誤嘅 JSON 檔案——修正之後，第二次就順利完成，模型亦可以正常使用。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.hk.png)


你亦可以向下捲動視覺化儀表板，查看狀態訊息同埋指標，如下所示：

| 訊息 | 指標 |
|:---|:---|
| ![訊息](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.hk.png) |  ![指標](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.hk.png)|


### 步驟 3.1：取得 ID 及在程式碼中測試微調後的模型


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### 步驟 3.2：喺 Playground 載入同測試你嘅微調模型

而家你可以用兩種方法測試你嘅微調模型。第一，你可以去 Playground，喺 Models 下拉選單揀返你新微調好嘅模型。另一個方法係用 Fine-tuning 面板入面嘅 "Playground" 選項（見上面截圖），會開一個 _比較_ 視圖，左右並排顯示基礎模型同微調模型，方便你快速評估。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.hk.png)

只需要填返你訓練數據用過嘅 system context，再輸入你想測試嘅問題。你會見到兩邊都會用同一個 context 同問題。執行比較之後，你就會見到兩個模型嘅回應有咩唔同。_留意微調模型會跟返你示例入面嘅格式去回應，而基礎模型就只係跟 system prompt_。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.hk.png)

你會發現比較畫面仲會顯示每個模型用咗幾多 token，同埋推理所需時間。**呢個例子其實好簡單，主要係示範流程，唔代表真實世界嘅數據或者情境**。你可能會見到兩個例子用咗一樣多嘅 token（因為 system context 同 user prompt 都一樣），但微調模型（自訂模型）推理時間會長啲。

喺現實應用入面，你唔會用咁簡單嘅例子，而係會用真實數據嚟微調（例如用產品目錄嚟做客戶服務），到時回應質素會更加明顯。喺 _呢啲_ 情況下，如果想用基礎模型達到同等回應質素，就要花多啲心思去設計 prompt，咁樣會用多啲 token，同時推理時間都可能會長咗。_如果你想試下，可以去 OpenAI Cookbook 睇下啲微調示例，開始動手做啦。_



---

**免責聲明**：  
本文件經由 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。我們致力確保翻譯準確，但請注意，自動翻譯可能會出現錯誤或不準確之處。原始語言的文件應被視為具權威性的來源。如涉及重要資訊，建議尋求專業人手翻譯。本翻譯所引致的任何誤解或誤釋，我們概不負責。
